In [16]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np
import sqlite3


In [2]:
def add_taxonomy(dataframe):
    result_values = {}
    for i in range(1, 16):
        codes_columns = f'Healthcare Provider Taxonomy Code_{i}'
        switch_columns = f'Healthcare Provider Primary Taxonomy Switch_{i}'
        
        # Check the condition and append values to the result list
        keys = dataframe['NPI'][dataframe[switch_columns]=='Y'].tolist()
        values = dataframe[codes_columns][dataframe[switch_columns]=='Y']
        for key, value in zip(keys,values):
            result_values[key] = value
            
    taxonomy_df = pd.DataFrame(list(result_values.items()), columns=['NPI', 'Taxonomy_Code'])
    nppes_merged = dataframe.merge(taxonomy_df, on = 'NPI', how = 'left')
    return nppes_merged

In [3]:
def Convert_strings_to_nan(value):
    try:
        return float(value)
    except ValueError:
        return np.nan

In [4]:
def fix_zipcode(series):
    # fill na with zeroes
    #change to int so the zeroes at the end gets removed
    # then change to strings and use zfill to add leading zeroes
    series = series.fillna(0).astype(float).astype(int).astype(str).str.zfill(5)
    # now i can split the strings at 5th item
    return series.str[:5]

In [5]:
columns_to_keep = ['NPI', 
                   'Entity Type Code', 
                   'Provider Organization Name (Legal Business Name)',
                   'Provider Last Name (Legal Name)',
                    'Provider First Name',
                    'Provider Middle Name',
                    'Provider Name Prefix Text',
                    'Provider Name Suffix Text',
                    'Provider Credential Text',       
                    'Provider First Line Business Practice Location Address',
                    'Provider Second Line Business Practice Location Address',
                    'Provider Business Practice Location Address City Name',
                    'Provider Business Practice Location Address State Name',
                    'Provider Business Practice Location Address Postal Code',
                    "Healthcare Provider Taxonomy Code_1",
                    "Healthcare Provider Primary Taxonomy Switch_1",
                    "Healthcare Provider Taxonomy Code_2",
                    "Healthcare Provider Primary Taxonomy Switch_2",
                    "Healthcare Provider Taxonomy Code_3",
                    "Healthcare Provider Primary Taxonomy Switch_3",
                    "Healthcare Provider Taxonomy Code_4",
                    "Healthcare Provider Primary Taxonomy Switch_4",
                    "Healthcare Provider Taxonomy Code_5",
                    "Healthcare Provider Primary Taxonomy Switch_5",
                    "Healthcare Provider Taxonomy Code_6",
                    "Healthcare Provider Primary Taxonomy Switch_6",
                    "Healthcare Provider Taxonomy Code_7",
                    "Healthcare Provider Primary Taxonomy Switch_7",
                    "Healthcare Provider Taxonomy Code_8",
                    "Healthcare Provider Primary Taxonomy Switch_8",
                    "Healthcare Provider Taxonomy Code_9",
                    "Healthcare Provider Primary Taxonomy Switch_9",
                    "Healthcare Provider Taxonomy Code_10",
                    "Healthcare Provider Primary Taxonomy Switch_10",
                    "Healthcare Provider Taxonomy Code_11",
                    "Healthcare Provider Primary Taxonomy Switch_11",
                    "Healthcare Provider Taxonomy Code_12",
                    "Healthcare Provider Primary Taxonomy Switch_12",
                    "Healthcare Provider Taxonomy Code_13",
                    "Healthcare Provider Primary Taxonomy Switch_13",
                    "Healthcare Provider Taxonomy Code_14",
                    "Healthcare Provider Primary Taxonomy Switch_14",
                    "Healthcare Provider Taxonomy Code_15",
                    "Healthcare Provider Primary Taxonomy Switch_15"
                    ]

In [6]:
cbsa = pd.read_csv('../data/ZIP_CBSA_122023.xlsx - Export Worksheet.csv')
taxonomy_code_classification =  pd.read_csv('../data/nucc_taxonomy_240.csv')

In [7]:
cbsa['zipcodes'] = fix_zipcode(cbsa['ZIP'])


In [8]:
hop = []
chunks = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', 
                              chunksize = 10000)
for chunk in chunks:
    chunk = chunk[chunk['transaction_count']>50]
    chunk = chunk[chunk['average_day_wait']<50]
    hop.append(chunk)
    
hop = pd.concat(hop, ignore_index = True)

In [9]:
chunks = []

for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', 
                         usecols=columns_to_keep, 
                         chunksize=10000):
    chunk_taxonomy = add_taxonomy(chunk)
    chunk_merged = pd.merge(left = chunk_taxonomy, right = taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), how ='left', left_on ='Taxonomy_Code', right_index = True)
    chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
    chunk_merged['zipcodes'] = fix_zipcode(chunk_merged['Provider Business Practice Location Address Postal Code'])
    chunk_merged_cbsa = pd.merge(left = chunk_merged, right = cbsa[['zipcodes','CBSA']].set_index('zipcodes'), how ='left', left_on ='zipcodes', right_index = True)
    chunk_merged_cbsa['CBSA'] = chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
    result_chunks.append(chunk_merged_cbsa)

# Concatenate all chunks into a single DataFrame
npi = pd.concat(chunks, ignore_index=True)

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (79,82,83,86,87,9

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  for chun

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. 

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,9

...

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,103,106) have mixed types. Specify dtype option 

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,9

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,71,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (79,82,83,86,87,90,9

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,79,82,83,86,87,90,91,94,95,98,99,102,103

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (32,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv',
/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_15840/3446724066.py:3: DtypeWarning: Columns (79,82,83,86,87,90,9

In [15]:
result_df.shape

(9016261, 48)

In [17]:
result_df.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy_Code,Classification,zipcodes,CBSA
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0,207X00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207X00000X,Orthopaedic Surgery,68847,28260
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X,Y,207RC0000X,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207RC0000X,Internal Medicine,32204,27260
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,251G00000X,"Hospice Care, Community Based",28304,22180
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000,0
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,174400000X,N,207RH0003X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207RH0003X,Internal Medicine,77090,26420


In [18]:
hop

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053
...,...,...,...,...,...,...
33563179,1417037664,1497939599,36,106,19.330,42.407
33563180,1417194903,1497939599,22,70,16.629,30.598
33563181,1417406372,1497939599,21,65,20.123,37.750
33563182,1417064825,1497940605,75,79,10.418,34.744


In [ ]:
# #db = sqlite3.connect('data/npi.sqlite')
# for chunk in pd.read_csv('data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv', 
#                           usecols= columns_to_keep,
#                               chunksize = 10000):
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left = chunk_taxonomy, 
#                             right = taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), 
#                             how = 'left',
#                             left_on = 'Taxonomy_Code',
#                             right_index = True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes']=fix_zipcode(
#         chunk_merged['Provider Business Practice Location Address Postal Code']
#         )
#     chunk_merged_cbsa = pd.merge(left = chunk_merged, 
#          right = cbsa[['zipcodes','CBSA']].set_index('zipcodes'), 
#          how = 'left',
#          left_on = 'zipcodes', 
#          right_index = True)
    
#     chunk_merged_cbsa['CBSA']= chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
    
#     chunk_merged_cbsa.to_sql('nppes', 
#                 db, 
#                 if_exists = 'append', 
#                 index = False)  

# #db.execute('CREATE INDEX NPI ON nppes(NPI)')
# db.close()

In [ ]:
# db = sqlite3.connect('../data/npi.sqlite')

# hop = []
# for chunk in pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize=10000):
#     chunk = chunk[(chunk['transaction_count'] > 50) & (chunk['average_day_wait'] < 50)]
#     hop.append(chunk)

# hop_df = pd.concat(hop, ignore_index=True)

# hop_df.to_sql('hop_npi', db, if_exists='replace', index=False)

# db.execute('CREATE INDEX IF NOT EXISTS from_npi ON hop_npi(from_npi)')
# db.close()

In [ ]:
# db = sqlite3.connect('../data/npi.sqlite')

# chunks = []
# for chunk in pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols=columns_to_keep, chunksize=10000):
#     chunk_taxonomy = add_taxonomy(chunk)
#     chunk_merged = pd.merge(left=chunk_taxonomy, right=taxonomy_code_classification[['Code', 'Classification']].set_index('Code'), how='left', left_on='Taxonomy_Code', right_index=True)
#     chunk_merged['Provider Business Practice Location Address Postal Code'] = chunk_merged['Provider Business Practice Location Address Postal Code'].apply(Convert_strings_to_nan)
#     chunk_merged['zipcodes'] = fix_zipcode(chunk_merged['Provider Business Practice Location Address Postal Code'])
#     chunk_merged_cbsa = pd.merge(left=chunk_merged, right=cbsa[['zipcodes', 'CBSA']].set_index('zipcodes'), how='left', left_on='zipcodes', right_index=True)
#     chunk_merged_cbsa['CBSA'] = chunk_merged_cbsa['CBSA'].fillna(0).astype(int)
#     chunks.append(chunk_merged_cbsa)

# npi = pd.concat(chunks, ignore_index=True)

# npi.to_sql('npi_nppes', db, if_exists='replace', index=False)

# db.execute('CREATE INDEX IF NOT EXISTS NPI ON npi_nppes(NPI)')
# db.close()